## First Steps

In [ ]:
# ================= MAIN CODE =================
# CONFIG DIR
webui_path = "/home/studio-lab-user/content/sdw"
root_path = "/home/studio-lab-user/content"
models_dir = f"{webui_path}/models/Stable-diffusion"
vaes_dir = f"{webui_path}/models/VAE"
embeddings_dir = f"{webui_path}/embeddings"
loras_dir = f"{webui_path}/models/Lora"
extensions_dir = f"{webui_path}/extensions"
control_dir = f"{webui_path}/models/ControlNet"
# ================ LIBRARIES ================     
!mkdir -p /home/studio-lab-user/content
dep_cycle_count = 15
import os
import re
import time
import json
import shutil
import subprocess
from datetime import timedelta
from subprocess import getoutput
from urllib.parse import unquote
from IPython.utils import capture
from IPython.display import clear_output
# ---
state_file = "/home/studio-lab-user/content/script_state.json"
def update_dependencies():
    print("Updating dependencies, may take some time....")
    !pip install -q --upgrade torchsde
    !pip install -q --upgrade pip
    !pip install -q --upgrade psutil
    !pip install -q gputil
    
    clear_output()
def get_script_state():
    if os.path.isfile(state_file):
        with open(state_file, "r") as f:
            state = json.load(f)
        return state
    else:
        return {"count": 0}
def increment_script_count():
    state = get_script_state()
    state["count"] += 1
    with open(state_file, "w") as f:
        json.dump(state, f)
state = get_script_state()
dep_cycle_count = max(1, min(dep_cycle_count, 50))
if state["count"] % dep_cycle_count == 0:
    update_dependencies()
increment_script_count()
# ================= OTHER =================
### Check if gpu or cpu
output = getoutput('nvidia-smi --query-gpu=gpu_name --format=csv')
if "name" in output:
    gpu_name = output[5:]
    clear_output()
    print(f'\r✅ \033[92;1mCurrent GPU:\033[0;1m {gpu_name}\033[97;1m', flush=True)
else:
    clear_output()
    print('\r\033[91;1m❎ ERROR: GPU - not detected. \nThe startup will be performed on the CPU.\n\033[97;1m', flush=True)
# ================ LIBRARIES ================     
flag_file = "/home/studio-lab-user/content/libraries_installed.txt"
if not os.path.exists(flag_file):
    print("Installing the libraries, may take some time...", end='')
    with capture.capture_output() as cap:
        %pip install -q opencv-python-headless huggingface-hub
        !pip install -q torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 torchtext==0.15.2 torchdata==0.6.1 --extra-index-url https://download.pytorch.org/whl/cu118 -U
        !pip install -q xformers==0.0.20 triton==2.0.0 -U     
        !pip install insightface
        %conda update -q -n base conda
        %conda install -q -y aria2
        %conda install -q -y glib
        %conda install -q zip
        with open(flag_file, "w") as f:
            f.write("hey ;3")
    del cap
    print("\rLibraries are installed!" + " "*30)
    time.sleep(2)
    clear_output()
# ================= MAIN CODE =================
if not os.path.exists(webui_path):
    action = "Updating/Installing WebUI and Extensions"
    start_install = int(time.time())
    print("⌚ Unpacking Stable Diffusion...", end='')
    with capture.capture_output() as cap:
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/godtrex99/mufiles/resolve/main/pikapika.zip -o repo.zip
        !unzip -q -o repo.zip -d /home/studio-lab-user/content
       # !mv /home/studio-lab-user/content/sdw/home/studio-lab-user/content/sdw /home/studio-lab-user/sdw
        !rm -rf repo.zip
        %cd {root_path}
        os.environ["SAFETENSORS_FAST_GPU"]='1'
        os.environ["CUDA_MODULE_LOADING"]="LAZY"
        os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
        os.environ["PYTHONWARNINGS"] = "ignore"
        #!mkfile -p {webui_path}/static/colabTimer.txt
        #!echo -n {start_colab} > {webui_path}/static/colabTimer.txt
    del cap
    install_time = timedelta(seconds=time.time()-start_install)
    print("\r🚀 Unpacking is complete! For","%02d:%02d:%02d ⚡\n" % (install_time.seconds / 3600, (install_time.seconds / 60) % 60, install_time.seconds % 60), end='', flush=True)
else:
    print("🚀 All unpacked... Updating WebUI. ⚡")
    %cd {webui_path}
    !git restore .
    !git pull -X theirs --rebase --autostash
    print("Updating Extensions. ⚡")
    %cd {extensions_dir}
    !{'for dir in /home/studio-lab-user/content/sdw/extensions/*/; do cd "$dir" && git fetch origin && git pull; done'}
    !cd /home/studio-lab-user/content/sdw/repositories/stable-diffusion-stability-ai && git restore .
    #start_colab = float(open(f'{webui_path}/static/colabTimer.txt', 'r').read())
    #time_since_start = str(timedelta(seconds=time.time()-start_colab)).split('.')[0]
    clear_output()
## List Models and stuff
#   print("\n\n\033[92mIf you don't see any downloaded files, enable the 'Detailed Downloads' feature in the widget.")
if any(not file.endswith('.txt') for file in os.listdir(models_dir)):
    print("\n\033[92m➤ Models\033[97m")
    !find {models_dir}/ -mindepth 1 ! -name '*.txt' -printf '%f\n'
if any(not file.endswith('.txt') for file in os.listdir(vaes_dir)):
    print("\n\033[92m➤ VAEs\033[97m")
    !find {vaes_dir}/ -mindepth 1 ! -name '*.txt' -printf '%f\n'
if any(not file.endswith('.txt') and not os.path.isdir(os.path.join(embeddings_dir, file)) for file in os.listdir(embeddings_dir)):
    print("\n\033[92m➤ Embeddings\033[97m")
    !find {embeddings_dir}/ -mindepth 1 -maxdepth 1 \( -name '*.pt' -or -name '*.safetensors' \) -printf '%f\n'
if any(not file.endswith('.txt') for file in os.listdir(loras_dir)):
    print("\n\033[92m➤ LoRAs\033[97m")
    !find {loras_dir}/ -mindepth 1 ! -name '*.keep' -printf '%f\n'
print(f"\n\033[92m➤ Extensions\033[97m")
!find {extensions_dir}/ -mindepth 1 -maxdepth 1 ! -name '*.txt' -printf '%f\n'
if any(not file.endswith(('.txt', '.yaml')) for file in os.listdir(control_dir)):
    print("\n\033[92m➤ ControlNet\033[97m")
    !find {control_dir}/ -mindepth 1 ! -name '*.yaml' -printf '%f\n' | sed 's/^[^_]*_[^_]*_[^_]*_\(.*\)_fp16\.safetensors$/\1/'
# Сleaning | A little memory clearing
with capture.capture_output() as cap:
    !find /home/studio-lab-user/ \( -name ".ipynb_checkpoints" -o -name ".aria2" \) -type d -exec rm -r {} \; >/dev/null 2>&1
    !rm -rf /home/studio-lab-user/.conda/envs/studiolab-safemode
    !rm -rf /home/studio-lab-user/.conda/envs/sagemaker-distribution
    !rm -rf /home/studio-lab-user/.conda/pkgs/cache
    !pip cache purge
    !rm -rf ~/.cache
del cap
##
#print(f"⌚️ You have been conducting this session for - \033[93;2m{time_since_start}\033[97m")
print(f"\r✨ {action} Completed!")

 ---

## Start UI

In [ ]:
ngrok_token = ""   # Get your ngrok_oken here : https://dashboard.ngrok.com/get-started/your-authtoken
#=================================================
#from colablib.colored_print import cprint, print_line
from IPython.display import clear_output
import GPUtil
import subprocess
import time
try:
    start_colab
except:
    start_colab = int(time.time())-5
def gpu_available():
    try:
        GPUs = GPUtil.getGPUs()
        return len(GPUs) > 0
    except Exception as e:
        return False
#echo -n {start_colab} > /home/studio-lab-user/content/sdw/static/colabTimer.txt
%cd /home/studio-lab-user/content/sdw
if gpu_available():
    print(f"[+] Running with GPU...")
    !python launch.py --no-download-sd --skip-install --listen --xformers --theme dark --enable-insecure-extension-access --disable-console-progressbars --no-half-vae --ngrok {ngrok_token}
else:
    print(f"[+] Running with CPU...")
    !python launch.py --no-download-sd --skip-torch-cuda-test --theme dark --precision full --enable-insecure-extension-access --ngrok {ngrok_token} --no-half --use-cpu SD GFPGAN BSRGAN ESRGAN SCUNet CodeFormer --all


---

## Storage Info

In [ ]:
# DISK/STORAGE INFO
more_info = False
# =======================================================================
import os
import psutil
directory = os.getcwd()
disk_space = psutil.disk_usage(directory) 
total = disk_space.total / (1024 ** 3)
used = disk_space.used / (1024 ** 3)
free = disk_space.free / (1024 ** 3)
print(f"Total storage: {total:.2f} GB \033[96m| \033[0mUse: {used:.2f} GB \033[96m| \033[0mFree: {free:.2f} GB")
if more_info:
    print("\033[96m---"*15 + "\n\033[96mMore Info:\n\033[0m")
    !du -h --max-depth=2 /home/studio-lab-user

## Auto Cleaner 

In [ ]:
# ~ AutoCleaner V3 ~
# =========================================================
import os
import time
import ipywidgets as widgets
from ipywidgets import Label, Button, VBox, HBox
from IPython.display import display, HTML
# Main Constants
webui_path = "/home/studio-lab-user/content/sdw"
directories = {
    "Images": f"{webui_path}/outputs",
    "Models": f"{webui_path}/models/Stable-diffusion/",
    "Vae": f"{webui_path}/models/VAE/",
    "LoRa": f"{webui_path}/models/Lora/",
    "ControlNet Models": f"{webui_path}/models/ControlNet/"
}
# ==================== CSS ====================
CSS_AC = """
<style>
/* General Styles */
hr {
    border-color: grey;
    background-color: grey;
    opacity: 0.25;
}
.instruction_AC {
    font-family: cursive;
    font-size: 18px;
    color: grey;
    user-select: none;
    cursor: default;
}
/* Container style */
.container_AC {
    background-color: #232323;
    width: 800px;
    padding: 15px;
    border-radius: 15px;
    box-shadow: 0 0 50px rgba(0, 0, 0, 0.3);
    margin-bottom: 5px;
    overflow: visible;
}
.container_AC::before {
    position: absolute;
    top: 5px;
    right: 10px;
    content: "AutoCleanerV3";
    font-weight: bold;
    font-size: 24px;
    color: rgba(0, 0, 0, 0.15);
}
.container_AC::after {
    position: absolute;
    bottom: 5px;
    right: 10px;
    content: "ANXETY";
    font-weight: bold;
    font-size: 16px;
    color: rgba(0, 0, 0, 0.15);
}
.custom-select-multiple_AC select {
    padding: 10px;
    font-family: cursive;
    border: 1px solid #262626;
    border-radius: 10px;
    color: white;
    background-color: #1c1c1c;
    box-shadow: inset 0 0 5px rgba(0, 0, 0, 0.5);
}
.output_AC {
    padding: 10px;
    height: auto;
    border: 1px solid #262626;
    border-radius: 10px;
    background-color: #1c1c1c;
    box-shadow: inset 0 0 5px rgba(0, 0, 0, 0.5);
    width: 100%;
}
.output_message_AC {
    font-family: cursive;
    color: white !important;
    font-size: 14px;
    user-select: none;
    cursor: default
}
/* Button style */
.button {
    width: auto;
    font-family: cursive;
    color: white !important;
    font-size: 14px;
    font-weight: bold;
    height: 35px;
    border-radius: 15px;
    background-image: radial-gradient(circle at top left, purple 10%, violet 90%);
    background-size: 200% 200%;
    background-position: left bottom;
    transition: background 0.5s ease-in-out, transform 0.3s ease;
}
.button:hover {
    cursor: pointer;
    background-size: 200% 200%;
    background-position: right bottom;
    transform: translateY(1px);
}
.button_execute_AC:hover {
    background-image: radial-gradient(circle at top left, purple 10%, #93ac47 90%);
}
.button_clear_AC:hover {
    background-image: radial-gradient(circle at top left, purple 10%, #fc3468 90%);
}
.button_execute_AC:active,
.button_clear_AC:active {
    filter: brightness(0.75);
}
.jupyter-widgets.lm-Widget:focus {
    outline: none;
}
/* Animation of elements */
/* Emergence */
.container_AC {
    animation-name: slideInTopBlur;
    animation-duration: 0.7s;
    animation-fill-mode: forwards;
}
@keyframes slideInTopBlur {
    0% {
        transform: translate3d(0, 50%, 0) scale(0.85) rotate3d(1, 0, 0, -85deg);
        filter: blur(5px) grayscale(1) brightness(0.5);
        opacity: 0;
    }
    100% {
        transform: translate3d(0, 0, 0) scale(1) rotate3d(1, 0, 0, 0deg);
        filter: blur(0) grayscale(0) brightness(1);
        opacity: 1;
    }
}
/* Leaving */
.container_AC.hide {
    animation-name: slideOutTopBlur;
    animation-duration: 0.5s;
    animation-fill-mode: forwards;
}
@keyframes slideOutTopBlur {
    0% {
        transform: translate3d(0, 0, 0) scale(1);
        filter: blur(0) grayscale(0) brightness(1);
        opacity: 1;
    }
    100% {
        transform: translate3d(0, -100%, 0);
        filter: blur(5px) grayscale(1) brightness(0);
        opacity: 0;
    }
}
</style>
"""
display(HTML(CSS_AC))
# ==================== CSS ====================
# ================ AutoCleaner function ================
def clean_directory(directory):
    deleted_files = 0
    for root, dirs, files in os.walk(directory):
        for file in files:
            if not file.endswith((".txt", ".yaml")):
                file_path = os.path.join(root, file)
                os.remove(file_path)
                deleted_files += 1
    return deleted_files
def get_word_variant(n, variants):
    unit = abs(n) % 10
    tens = abs(n) % 100
    if tens in range(11, 15): 
        return variants[2] 
    elif unit == 1: 
        return variants[0]
    elif unit in range(2, 5): 
        return variants[1]
    return variants[2]
def on_execute_button_press(button):
    selected_cleaners = auto_cleaner_widget.value
    deleted_files_dict = {}
    for option in selected_cleaners:
        if option in directories:
            deleted_files_dict[option] = clean_directory(directories[option])
    output.clear_output()
    
    with output:
        for message in generate_messages(deleted_files_dict):
            message_widget = HTML(f'<p class="output_message_AC">{message}</p>')
            display(message_widget)
            
def on_clear_button_press(button):
    container.add_class("hide")
    time.sleep(0.3)
    widgets.Widget.close_all()
def generate_messages(deleted_files_dict):
    messages = []
    word_variants = {
        "Images": ["Image", "Images", "Images"],
        "Models": ["Model", "Models", "Models"],
        "Vae": ["VAE", "VAE", "VAE"],
        "LoRa": ["LoRa", "LoRa", "LoRa"],
        "ControlNet Models": ["ControlNet Model", "ControlNet Models", "ControlNet Models"]
    }
    deleted_word_variants = ["Deleted", "Deleted", "Deleted"]
    for key, value in deleted_files_dict.items():
        word_variant = word_variants.get(key, ["", "", ""])
        deleted_word = get_word_variant(value, deleted_word_variants)
        object_word = get_word_variant(value, word_variant)
        messages.append(f"{deleted_word} {value} {object_word}")
    return messages
# ================ AutoCleaner function ================
# UI Code
AutoCleaner_options = AutoCleaner_options = list(directories.keys())
instruction_label = widgets.HTML('''
<span class="instruction_AC">Use <span style="color: #B2B2B2;">ctrl</span> or <span style="color: #B2B2B2;">shift</span> for multiple selections.</span>
''')
auto_cleaner_widget = widgets.SelectMultiple(options=AutoCleaner_options, layout=widgets.Layout(width='100%')).add_class("custom-select-multiple_AC")
output = widgets.Output().add_class("output_AC")
# ---
execute_button = Button(description='Execute Cleaning').add_class("button_execute_AC").add_class("button")
execute_button.on_click(on_execute_button_press)
clear_button = Button(description='Hide Widget').add_class("button_clear_AC").add_class("button")
clear_button.on_click(on_clear_button_press)
# ---
buttons = widgets.HBox([execute_button, clear_button])
container = VBox([instruction_label, widgets.HTML('<hr>'), auto_cleaner_widget, output, widgets.HTML('<hr>'), buttons]).add_class("container_AC")
display(container)